# Exercise - RNN Classification

In this notebook, we will perform a classification task using RNNs (i.e., a sequence to value prediction). We have hourly power consumption of households for 12 hours. Based on this, we will determine whether the power grid is strained (1) or not (0). 

Therefore, use the columns from `Hour 0` to `Hour 11` to predict the `target` column in the `power.csv` data set.

Hint1: Use Tutorial 1 for help.

Hint2: Don't forget to adjust the number of neurons in the input layers correctly. Otherwise, you will run into errors.

In [53]:
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import mean_squared_error


# Common imports
import numpy as np
import os
import pandas as pd

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)



# Read the Dataset

In [54]:
power = pd.read_csv('power.csv')

power.head()

,Hour 0,Hour 1,Hour 2,Hour 3,Hour 4,Hour 5,Hour 6,Hour 7,Hour 8,Hour 9,Hour 10,Hour 11,target
0,2.550633,2.523400,2.582333,2.541667,2.475733,2.476233,2.455800,2.447200,2.441733,3.146133,2.661733,2.576000,1
1,1.596933,1.619567,2.473733,2.731133,2.431133,2.479667,1.690200,1.332133,1.375167,1.050900,0.585900,2.651900,1
2,0.534933,0.540467,0.575367,0.526500,0.521900,0.565333,1.426467,0.602067,0.547433,0.525067,1.270300,0.393767,0
3,1.085867,0.651233,0.634600,0.653000,0.646067,0.628400,0.611067,0.612533,0.660100,0.606067,1.471867,0.834533,0
4,0.456000,0.286300,0.310833,0.250933,0.277667,0.308633,0.610400,1.563533,1.421867,3.324400,3.207567,1.425433,1


In [55]:
power.shape

(1417, 13)

# Split the Data



In [56]:
# First 1000 days are for train
train = power.iloc[:1000]

# Remaining 417 days are for test
test = power.iloc[-417:]

In [57]:
train.shape

(1000, 13)

In [58]:
test.shape

(417, 13)

# Create Input and Target values

The first 12 columns (hourly data) will be input to predict the last column (i.e., target)

In [59]:
# The first 12 columns (from 0 to 11) are inputs

train_inputs = train.iloc[:,:12]

## Add one more dimension to make it ready for RNNs

In [60]:
#Create an additional dimension for train

train_x = np.array(train_inputs).reshape(1000,12,1)

train_x.shape 

(1000, 12, 1)

## Set the target

In [61]:
# The last column is TARGET

train_target = train.iloc[:,-1]

## Repeat for TEST

In [62]:
test.shape

(417, 13)

In [63]:
# The first 12 columns are inputs

test_inputs = test.iloc[:,:12]

In [64]:
#Create an additional dimension for test

test_x = np.array(test_inputs).reshape(417,12,1)

test_x.shape 

(417, 12, 1)

In [65]:
# The last column is TARGET

test_target = test.iloc[:,-1]

# Build a normal (cross-sectional) NN

This model assumes that the data is NOT a time-series data set. It treats the data as cross-sectional and the columns being independent of each other.

In [66]:
model = keras.models.Sequential([
    
    keras.layers.Flatten(input_shape=[12, 1]),
    keras.layers.Dense(12, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
    
])

In [67]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = tf.keras.optimizers.Nadam(learning_rate=0.01)

# If multiclass, use "sparse_categorical_crossentropy" as the loss function
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])


history = model.fit(train_x, train_target, epochs=20,
                    validation_data=(test_x, test_target))

Epoch 1/20
32/32 [==============================] - 2s 9ms/step - loss: 0.6468 - accuracy: 0.6100 - val_loss: 0.6251 - val_accuracy: 0.6787
Epoch 2/20
32/32 [==============================] - 0s 3ms/step - loss: 0.5778 - accuracy: 0.7290 - val_loss: 0.5717 - val_accuracy: 0.6978
Epoch 3/20
32/32 [==============================] - 0s 3ms/step - loss: 0.5209 - accuracy: 0.7430 - val_loss: 0.5621 - val_accuracy: 0.7170
Epoch 4/20
32/32 [==============================] - 0s 3ms/step - loss: 0.4970 - accuracy: 0.7500 - val_loss: 0.5269 - val_accuracy: 0.7074
Epoch 5/20
32/32 [==============================] - 0s 3ms/step - loss: 0.4743 - accuracy: 0.7640 - val_loss: 0.5244 - val_accuracy: 0.7074
Epoch 6/20
32/32 [==============================] - 0s 3ms/step - loss: 0.4709 - accuracy: 0.7460 - val_loss: 0.5359 - val_accuracy: 0.6954
Epoch 7/20
32/32 [==============================] - 0s 3ms/step - loss: 0.4643 - accuracy: 0.7540 - val_loss: 0.5173 - val_accuracy: 0.7338
Epoch 8/20
32/32 [==

In [68]:
# evaluate the model

scores = model.evaluate(test_x, test_target, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.535049319267273, 0.7122302055358887]

In [69]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.54
accuracy: 71.22%


# Build a simple RNN with one layer

In [96]:
n_steps = 12
n_inputs = 1


model = keras.models.Sequential([
    
    keras.layers.SimpleRNN(12, input_shape=[n_steps, n_inputs]),
    keras.layers.Dense(1, activation='sigmoid')
])

In [97]:
from tensorflow.keras.callbacks import EarlyStopping


earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')

callback = [earlystop]

In [98]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = tf.keras.optimizers.Nadam(learning_rate=0.01)

# If multiclass, use "sparse_categorical_crossentropy" as the loss function
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])


history = model.fit(train_x, train_target, epochs=50,
                    validation_data=(test_x, test_target), callbacks=callback)

Epoch 1/50
32/32 [==============================] - 3s 15ms/step - loss: 0.6080 - accuracy: 0.6700 - val_loss: 0.5513 - val_accuracy: 0.7314
Epoch 2/50
32/32 [==============================] - 0s 5ms/step - loss: 0.5005 - accuracy: 0.7410 - val_loss: 0.5918 - val_accuracy: 0.7002
Epoch 3/50
32/32 [==============================] - 0s 5ms/step - loss: 0.4995 - accuracy: 0.7420 - val_loss: 0.5758 - val_accuracy: 0.7074
Epoch 4/50
32/32 [==============================] - 0s 5ms/step - loss: 0.4930 - accuracy: 0.7540 - val_loss: 0.5243 - val_accuracy: 0.7266
Epoch 5/50
32/32 [==============================] - 0s 5ms/step - loss: 0.4821 - accuracy: 0.7570 - val_loss: 0.5354 - val_accuracy: 0.7146
Epoch 6/50
32/32 [==============================] - 0s 5ms/step - loss: 0.4858 - accuracy: 0.7560 - val_loss: 0.5504 - val_accuracy: 0.7050
Epoch 7/50
32/32 [==============================] - 0s 5ms/step - loss: 0.4873 - accuracy: 0.7570 - val_loss: 0.5072 - val_accuracy: 0.7506
Epoch 8/50
32/32 [=

In [100]:
# evaluate the model

scores = model.evaluate(test_x, test_target, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.5214211940765381, 0.7314148545265198]

In [101]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.52
accuracy: 73.14%


In [102]:
# Predictions are probabilities.

predictions = model.predict(test_x)

14/14 [==============================] - 0s 2ms/step


# Build a simple RNN with two or more layers

In [103]:
n_steps = 12
n_inputs = 1


model = keras.models.Sequential([
    keras.layers.SimpleRNN(12, return_sequences=True, input_shape=[n_steps, n_inputs] ),
    keras.layers.SimpleRNN(12, return_sequences=True),
    keras.layers.SimpleRNN(12), 
    keras.layers.Dense(1, activation='sigmoid')
])


In [104]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_target, epochs=20,
                   validation_data = (test_x, test_target), callbacks=callback)

Epoch 1/20
32/32 [==============================] - 5s 26ms/step - loss: 0.5569 - accuracy: 0.7010 - val_loss: 0.5503 - val_accuracy: 0.7506
Epoch 2/20
32/32 [==============================] - 0s 9ms/step - loss: 0.4892 - accuracy: 0.7510 - val_loss: 0.5650 - val_accuracy: 0.6978
Epoch 3/20
32/32 [==============================] - 0s 10ms/step - loss: 0.4774 - accuracy: 0.7550 - val_loss: 0.7805 - val_accuracy: 0.6115
Epoch 4/20
32/32 [==============================] - 0s 14ms/step - loss: 0.5001 - accuracy: 0.7400 - val_loss: 0.5344 - val_accuracy: 0.7194
Epoch 5/20
32/32 [==============================] - 0s 10ms/step - loss: 0.4617 - accuracy: 0.7810 - val_loss: 0.5311 - val_accuracy: 0.7146
Epoch 6/20
32/32 [==============================] - 0s 10ms/step - loss: 0.4558 - accuracy: 0.7720 - val_loss: 0.4934 - val_accuracy: 0.7530
Epoch 7/20
32/32 [==============================] - 0s 10ms/step - loss: 0.4393 - accuracy: 0.7800 - val_loss: 0.4862 - val_accuracy: 0.7506
Epoch 8/20
32/

In [105]:
# evaluate the model

scores = model.evaluate(test_x, test_target, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.5084664225578308, 0.7290167808532715]

In [106]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.51
accuracy: 72.90%


# Build a LSTM with one layer

In [107]:
n_steps = 12
n_inputs = 1

model = keras.models.Sequential([
    
    keras.layers.LSTM(12, input_shape=[n_steps, n_inputs]),
    keras.layers.Dense(1, activation='sigmoid')
])

In [108]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_target, epochs=50,
                   validation_data = (test_x, test_target), callbacks=callback)

Epoch 1/50
32/32 [==============================] - 4s 29ms/step - loss: 0.6140 - accuracy: 0.6520 - val_loss: 0.5628 - val_accuracy: 0.7098
Epoch 2/50
32/32 [==============================] - 0s 8ms/step - loss: 0.4927 - accuracy: 0.7470 - val_loss: 0.5599 - val_accuracy: 0.6954
Epoch 3/50
32/32 [==============================] - 0s 8ms/step - loss: 0.4783 - accuracy: 0.7540 - val_loss: 0.5776 - val_accuracy: 0.7074
Epoch 4/50
32/32 [==============================] - 0s 9ms/step - loss: 0.4787 - accuracy: 0.7640 - val_loss: 0.5254 - val_accuracy: 0.7146
Epoch 5/50
32/32 [==============================] - 0s 8ms/step - loss: 0.4639 - accuracy: 0.7620 - val_loss: 0.5211 - val_accuracy: 0.7146
Epoch 6/50
32/32 [==============================] - 0s 9ms/step - loss: 0.4676 - accuracy: 0.7570 - val_loss: 0.5227 - val_accuracy: 0.7098
Epoch 7/50
32/32 [==============================] - 0s 8ms/step - loss: 0.4636 - accuracy: 0.7550 - val_loss: 0.5130 - val_accuracy: 0.7530
Epoch 8/50
32/32 [=

In [109]:
# evaluate the model

scores = model.evaluate(test_x, test_target, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.5091004371643066, 0.7266187071800232]

In [110]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.51
accuracy: 72.66%


# Build a LSTM with two or more layers

In [111]:
n_steps = 12
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.LSTM(12, return_sequences=True, input_shape=[n_steps, n_inputs]),
    keras.layers.LSTM(12, return_sequences=True),
    keras.layers.LSTM(12),
    keras.layers.Dense(1, activation='sigmoid')
])

In [85]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_target, epochs=20,
                   validation_data = (test_x, test_target), callbacks=callback)

Epoch 1/20
32/32 [==============================] - 11s 68ms/step - loss: 0.6238 - accuracy: 0.6490 - val_loss: 0.6476 - val_accuracy: 0.6811
Epoch 2/20
32/32 [==============================] - 1s 24ms/step - loss: 0.5247 - accuracy: 0.7340 - val_loss: 0.6347 - val_accuracy: 0.6787
Epoch 3/20
32/32 [==============================] - 1s 26ms/step - loss: 0.4981 - accuracy: 0.7500 - val_loss: 0.7459 - val_accuracy: 0.5923
Epoch 4/20
32/32 [==============================] - 1s 25ms/step - loss: 0.5004 - accuracy: 0.7380 - val_loss: 0.5039 - val_accuracy: 0.7170
Epoch 5/20
32/32 [==============================] - 1s 20ms/step - loss: 0.4679 - accuracy: 0.7660 - val_loss: 0.5416 - val_accuracy: 0.7170
Epoch 6/20
32/32 [==============================] - 1s 21ms/step - loss: 0.4736 - accuracy: 0.7500 - val_loss: 0.5031 - val_accuracy: 0.7146
Epoch 7/20
32/32 [==============================] - 1s 21ms/step - loss: 0.4638 - accuracy: 0.7540 - val_loss: 0.4980 - val_accuracy: 0.7530
Epoch 8/20
3

In [86]:
# evaluate the model

scores = model.evaluate(test_x, test_target, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.5331521034240723, 0.7482014298439026]

In [87]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.53
accuracy: 74.82%


# Build a GRU with one layer

In [112]:
n_steps = 12
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.GRU(12, input_shape=[n_steps, n_inputs]),
    keras.layers.Dense(1, activation='sigmoid')
])

In [89]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_target, epochs=20,
                   validation_data = (test_x, test_target), callbacks=callback)

Epoch 1/20
32/32 [==============================] - 5s 29ms/step - loss: 0.5529 - accuracy: 0.7120 - val_loss: 0.5602 - val_accuracy: 0.7314
Epoch 2/20
32/32 [==============================] - 0s 9ms/step - loss: 0.4763 - accuracy: 0.7480 - val_loss: 0.5628 - val_accuracy: 0.6978
Epoch 3/20
32/32 [==============================] - 0s 9ms/step - loss: 0.4730 - accuracy: 0.7440 - val_loss: 0.6590 - val_accuracy: 0.6619
Epoch 4/20
32/32 [==============================] - 0s 9ms/step - loss: 0.4773 - accuracy: 0.7610 - val_loss: 0.5424 - val_accuracy: 0.7122
Epoch 5/20
32/32 [==============================] - 0s 9ms/step - loss: 0.4674 - accuracy: 0.7570 - val_loss: 0.5283 - val_accuracy: 0.7074
Epoch 6/20
32/32 [==============================] - 0s 9ms/step - loss: 0.4684 - accuracy: 0.7520 - val_loss: 0.5127 - val_accuracy: 0.7122
Epoch 7/20
32/32 [==============================] - 0s 9ms/step - loss: 0.4625 - accuracy: 0.7560 - val_loss: 0.5007 - val_accuracy: 0.7434
Epoch 8/20
32/32 [=

In [90]:
# evaluate the model

scores = model.evaluate(test_x, test_target, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.5144416689872742, 0.7362110018730164]

In [91]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.51
accuracy: 73.62%


# Build a GRU with two or more layers

In [116]:
n_steps = 12
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.GRU(12, return_sequences=True, input_shape=[n_steps, n_inputs]),
    keras.layers.GRU(12, return_sequences=True),
    keras.layers.GRU(12),
    keras.layers.Dense(1, activation='sigmoid')
])

In [117]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_target, epochs=20,
                   validation_data = (test_x, test_target), callbacks=callback)

Epoch 1/20
32/32 [==============================] - 12s 68ms/step - loss: 0.5701 - accuracy: 0.7050 - val_loss: 0.5602 - val_accuracy: 0.7338
Epoch 2/20
32/32 [==============================] - 1s 22ms/step - loss: 0.5010 - accuracy: 0.7340 - val_loss: 0.5623 - val_accuracy: 0.7050
Epoch 3/20
32/32 [==============================] - 1s 22ms/step - loss: 0.4840 - accuracy: 0.7480 - val_loss: 0.6450 - val_accuracy: 0.6379
Epoch 4/20
32/32 [==============================] - 1s 22ms/step - loss: 0.5004 - accuracy: 0.7460 - val_loss: 0.5218 - val_accuracy: 0.6978
Epoch 5/20
32/32 [==============================] - 1s 22ms/step - loss: 0.4694 - accuracy: 0.7520 - val_loss: 0.5569 - val_accuracy: 0.7122
Epoch 6/20
32/32 [==============================] - 1s 23ms/step - loss: 0.4714 - accuracy: 0.7500 - val_loss: 0.5189 - val_accuracy: 0.7218
Epoch 7/20
32/32 [==============================] - 1s 21ms/step - loss: 0.4619 - accuracy: 0.7630 - val_loss: 0.4993 - val_accuracy: 0.7482
Epoch 8/20
3

In [118]:
# evaluate the model

scores = model.evaluate(test_x, test_target, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.5419779419898987, 0.7050359845161438]

In [119]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.54
accuracy: 70.50%
